# Práctica Final: Analisis Descriptivo de Datos






Te eligieron como el tecnico del equipo mas grande de Bolivia: [Bolivar](https://en.wikipedia.org/wiki/Club_Bol%C3%ADvar) y como primer paso para mejorar el equipo, debes realizar un analisis de un conjunto de jugadores para realizar contrataciones mas adelante.
Una forma de evaluar la calidad del jugador seria viendo los juegos en los que el participa. Tu crees que viendo las *habilidades fisicas* de los candidatos como *correr* y *saltar* pueden ser una buena forma de medir las habilidades de un jugador de futbol. 

Debido a que pasaste el modulo 6 del Diplomado Data Analytics, realizaras un analisis del conjunto de datos. Trabajaras con el dataset de los jugadorres del video juego [FIFA 19](https://www.ea.com/games/fifa/fifa-19), que contiene datos de miles de jugares de futbol, en particular sus habilidades en diferentes campos.

Cada fila del dataset corresponde a un jugador (vas a reconocerlos). Contiene informacion como la nacionalidad del jugador y su valor en el mercado. Los atributos que se refieren a las habilidades de los jugadores, la mayoria son positivos en diferentes rangos, donde *mayor valor* corresponde a *el jugador siendo mejor* en esa habilidad.


### A1. Conociendo los datos

#### A1.1.
Carrga el dataset `fifa19_players.csv` en un Pandas DataFrame


In [2]:
''' Aca escribe el codigo'''
%matplotlib inline
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('adult_train.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
df.shape


(32561, 15)

In [5]:
df['fnlwgt'].unique().shape

(21648,)

In [6]:
df['education'].unique()

array([' Bachelors', ' HS-grad', ' 11th', ' Masters', ' 9th',
       ' Some-college', ' Assoc-acdm', ' Assoc-voc', ' 7th-8th',
       ' Doctorate', ' Prof-school', ' 5th-6th', ' 10th', ' 1st-4th',
       ' Preschool', ' 12th'], dtype=object)

In [7]:
df['workclass'].unique()

array([' State-gov', ' Self-emp-not-inc', ' Private', ' Federal-gov',
       ' Local-gov', ' ?', ' Self-emp-inc', ' Without-pay',
       ' Never-worked'], dtype=object)

In [8]:
df['fnlwgt'].unique().shape

(21648,)

In [9]:
df.shape

(32561, 15)

#### A1.2.
La columna *Overall* contiene una calificacion que representa el rendimiento general del jugador de futbol. Antes de empezar, quieres estar seguro que hay la suficiente diversidad en el dataset para obtener conclusiones significatiivas (e.g. no deberia ser el caso que todos los jugadores tienen el mismo nivel de habilidades) 
* Grafica el histograma con los scores y analiza el plot. Que tipo de distribucion tenemos?


In [10]:
plt.hist(df['Overall'], bins=20)

KeyError: 'Overall'

In [ ]:
df['Overall'].describe()


In [ ]:
df_first_3=df[['Name','Age','Nationality', 'Overall','Release Clause']].head(3) 
print(df_first_3) 

In [ ]:
df_last_3=df[['Name','Age','Nationality', 'Overall','Release Clause']].tail(3) 
print(df_last_3) 

### Respuesta A1. Que tipo de distribucion tenemos? 

La distribucion se asemeja a la normal, la media es de 66,con una desviacion estandar de 6.9.

Es interesante analizar los datos minimos y maximos, el overall maximo es de 94 que pertenece a las estrellas como Cristiano Ronaldo o Lionel Messi, en tercer Lugar tenemos a Neymar Junior.

El overall Minimo corresponde a jovenes jugadores con una media de edad de 16 anhos al inicio de su carrera, con un overall de 46 u 47 . Estos tres ultimos son de Inglaterra.

### A2. Preprocesamiento y Analisis

Los atributos que usaras para las siguientes tareas son los siguientes: (puedes remover el resto de las columnas)


In [ ]:
skills = ['Name','Age','Nationality','Position','Club','Value','International Reputation', 'Work Rate',
          'Finishing','Dribbling','Acceleration','SprintSpeed', 'Agility', 'Reactions', 'Balance',
          'Jumping', 'Stamina', 'Strength','Overall']

In [ ]:
df_skills=df[['Name','Age','Nationality','Position','Club','Value','International Reputation', 'Work Rate',
          'Finishing','Dribbling','Acceleration','SprintSpeed', 'Agility', 'Reactions', 'Balance',
          'Jumping', 'Stamina', 'Strength','Overall']]
df_skills.head()

#### A2.1.

Más adelante queremos aplicar modelos predictivos, para ello debemos procesar nuestros datos y convertir algunos atributos de habilidades en valores *numericos*.
'Work Rate' se refiere a cuanto esfuerzo le pone el jugador a su juego cuando ataca y defiende respectivamente, pero no es numerico. Transformar el atributo **Work Rate** en _dos_ atributos numericos (dos columnas)


In [ ]:
WR = df_skills['Work Rate'].str.split('/', expand=True)
WR.columns = ['Attack_Rate', 'Defend_Rate']
df_skills = pd.concat([df_skills, WR], axis=1)
df_skills


In [ ]:
''' Aca escribe el codigo'''
df_skills['Attack_Rate'] = df_skills['Attack_Rate'].map({'Low': 0,
                             'Medium':1 ,
                             'High': 2},
                             na_action=None)
df_skills['Defend_Rate'] = df_skills['Defend_Rate'].map({' Low': 0,
                             ' Medium':1 ,
                             ' High': 2},
                             na_action=None)
df_skills


In [ ]:
df_skills_top_10=df_skills[['Name','Attack_Rate', 'Defend_Rate']].head(10) 
print(df_skills_top_10) 

#### A2.2.

Como en todos los datasets de aplicaciones reales, encontraras *valores faltantes*. Hay varias formas de *rellenarlos* viendo los otros records para los cuales estos valores no faltan.

* Use una *simple agregacion* de los records para los cuales los valores estan presentes (no faltantes) para llenar en los valores faltantes. Unicamente debes realizar esta agregacion para los atibutos **numericos**, para los demas atributos puedes eliminar esas muestras

* Existe algun outlier/anomalia en alguno de los atributos? Incluye el analisis para identificarlos

In [ ]:
df_skills.dtypes

In [ ]:
''' Aca escribe el codigo'''
df_skills.isnull().sum()

### Para llenar los datos faltantes en Sprint Speed
Se utilizaron los datos de la columna Agilidad, al ser similares a Sprint Speed.


In [ ]:
df_skills['SprintSpeed'].fillna(df_skills['Agility'], inplace=True)
df_skills


In [ ]:
df_skills.isnull().sum()

In [ ]:
df_skills.shape

In [ ]:
df_skills_clean=df_skills.dropna( how="any")
df_skills_clean

In [ ]:
df_skills_clean.isnull().sum()

In [ ]:
df_skills_clean.shape

In [ ]:
df_skills_clean.dtypes

In [ ]:
df_skills_clean.columns

### Existe algun outlier/anomalia en alguno de los atributos? Incluye el analisis para identificarlos
Se puede identificar los outliers por atributos numericos. A continuacion se describen los principales atributos:
#### Edad
En el caso de la edad, la edad maxima en este dataset es de 45 anhos, y la minima es de 16. La media es de 25 anhos.



In [ ]:
df_skills_clean.describe(exclude=[object])

#### A2.3.

Queremos realizar un ranking de los clubes basados en: 'Value', 'International Reputation' y 'Overall' de todos los jugadores que pertenecen a un mismo club ('Club').
- Muestra en una tabla el top 10 de clubes basado en la suma de 'Value' de todos sus jugadores. Que club tiene mas valor? que club el menor valor?
- Muestra en una tabla el top 10 de clubes basado en la suma de 'International Reputation' de todos sus jugadores. 
- Muestra en una tabla el top 10 de clubes basado en la suma de 'Overall' de todos sus jugadores

Que club/clubes se encuentran en los tres rankings entre el top 10?

In [ ]:
''' Aca escribe el codigo'''
dfV=df_skills_clean.groupby(['Club'])['Value'].sum().reset_index(name="Value Club").sort_values(by=['Value Club'],ascending=False).head(10)
dfV

In [ ]:
dfIR=df_skills_clean.groupby(['Club'])['International Reputation'].sum().reset_index(name="IR Club").sort_values(by=['IR Club'],ascending=False).head(10)
dfIR

In [ ]:
dfO=df_skills_clean.groupby(['Club'])['Overall'].sum().reset_index(name="Overall Club").sort_values(by=['Overall Club'],ascending=False)
dfO

#### A2.4.

Queremos realizar una comparacion justa entre clubes para el Overall debemos dividir el total obtenido entre el numero de jugadores pertenecientes al club.

- Muestra en una tabla el top 10 de clubes basado en el valor 'normalizado' de 'Overall' de todos sus jugadores
- Cual es el mejor club basado en este ranking? Cual es el peor club basado en este ranking?

In [ ]:
''' Aca escribe el codigo'''
dfNP=df_skills_clean.groupby(['Club'])['Name'].count().reset_index(name="Número de jugadores").sort_values(by=['Número de jugadores'],ascending=False)
dfNP

In [ ]:
dfNP.tail(10)

In [ ]:
dfO['Indice Normalizado']=dfO['Overall Club']/dfNP['Número de jugadores']
dfO.sort_values(by='Indice Normalizado',ascending=False).head(10)

In [ ]:
dfO.sort_values(by='Indice Normalizado',ascending=True).head(10)

#### A2.5.

Ahora realizamos la misma comparacion pero a nivel Internacional. Cual es el top 10, en base al Overall de los jugadores basados en su nacionalidad?

- Muestra en una tabla el top 10 de selecciones nacionales basado en el valor 'normalizado' de 'Overall' de todos sus jugadores
- Cual es el mejor pais basado en este ranking? Cual es el peor pais basado en este ranking? 
- Cual es el Overall de la seleccion de Bolivia?

In [ ]:
''' Aca escribe el codigo'''
dfON=df_skills_clean.groupby(['Nationality'])['Overall'].sum().reset_index(name="Overall Country").sort_values(by=['Overall Country'],ascending=False)
dfON

In [ ]:
dfNPN=df_skills_clean.groupby(['Nationality'])['Name'].count().reset_index(name="Número de jugadores").sort_values(by=['Número de jugadores'],ascending=False)
dfNPN

In [ ]:
dfON['Indice Normalizado']=dfON['Overall Country']/dfNPN['Número de jugadores']
dfON.sort_values(by='Indice Normalizado',ascending=False).head(10)

In [ ]:
dfON.sort_values(by='Indice Normalizado',ascending=True).head(10)

In [ ]:
dfON.sort_values(by='Nationality',ascending=True).head(18)

In [ ]:
dfON.loc[dfON['Nationality']=='Bolivia']

#### A2.6.

Queremos entender como es la distribucion de edades de acuerdo a las diferentes posiciones 'Position' en las que juegan

- En que posicion son jugadores de mayor edad? 
- En que posicion son jugadores de menor edad?

In [ ]:
''' Aca escribe el codigo'''
dfPN=df_skills_clean.groupby(['Position'])['Name'].count().reset_index(name="Número de jugadores").sort_values(by=['Número de jugadores'],ascending=False)
dfPN

In [ ]:
dfA=df_skills_clean.groupby(['Position'])['Age'].sum().reset_index(name="A").sort_values(by=['A'],ascending=False)
dfA

In [ ]:
dfPN['Edad Promedio']=dfA['A']/dfPN['Número de jugadores']
dfPN.sort_values(by='Edad Promedio',ascending=False).head(10)

In [ ]:
dfPN.sort_values(by='Edad Promedio',ascending=True).head(10)

### B1. Relacion entre atributos


#### B1.1.

Realiza un analisis de correlacion entre los diferentes atributos de cada jugador. ** No es necesario que realices el analisis con todos los atributos, puedes seleccionar los que te parezcan mas relevantes.

- Cuales son los atributos que presentan una mayor correlacion?

In [ ]:
''' Aca escribe el codigo'''
df_skills_clean.corr()

In [ ]:
import seaborn as sns
corrmat=df_skills_clean.corr()
sns.clustermap(corrmat, cmap="RdBu")

#### B1.2.

Los atributos que se refieren a las habilidades de los jugadores son:


In [ ]:
athetic_skills = ['Work Rate','Finishing','Dribbling','Acceleration','SprintSpeed', 'Agility', 'Reactions', 'Balance','Jumping', 'Stamina', 'Strength','Overall']

Para Work Rate, debes usar las dos columnas que creaste en el ejercicio A2.1

Para decidir que jugador contratar, necesitas saber como estas habilidades se transforman al rendimiento en el futbol.

Para modelar el rendimiento en futbol ('Overall') para un jugador utilizaras "Regresion Lineal". Recordemos que la Regresion Lineal minimiza:




$$||y-X\beta||_2^2,$$

donde $y$ es un vector de todas las puntuaciones, $X$ es la matriz con los datos, cada *fila* son los atributos de cada jugador, $\beta$ son los parametros beta


Realiza un Analisis de Regresion Lineal en base a los atributos: Reactions, Agility, defense_work_rate


In [ ]:
%matplotlib inline
import math
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
mod = smf.ols(formula='Overall ~ (Reactions) + (Agility) + C(Defend_Rate)', data=df_skills_clean)
np.random.seed(3)
res = mod.fit()

In [ ]:
print(res.summary())

Realizar el mismo analisis, pero incluyendo Finishing en lugar de Agility. Los atributos son: Reactions, Finishing, defense_work_rate. Comparar los valores $R^2$ de ambos analisis

In [ ]:
df_skills_clean['Defend_Rate'].unique()

In [ ]:
mod = smf.ols(formula='Overall ~ (Reactions) + (Defend_Rate) + (Finishing)', data=df_skills_clean)
res = mod.fit()
print(res.summary())

In [ ]:
mod = smf.ols(formula='Overall ~ (Reactions)*C(Defend_Rate ,Treatment(reference=0)) + (Finishing)*C(Defend_Rate ,Treatment(reference=0))', data=df_skills_clean)
res = mod.fit()
print(res.summary())

In [ ]:
coefficients = res.params.values
print(coefficients)

### B3. Pandas Profiling/SweetViz

Utiliza la herramienta Pandas Profiling o Sweetviz para realizar un analisis descriptivo de datos.

Escribe algunas conclusiones del reporte generado por la libreria

In [ ]:
import pandas_profiling
df_skills_clean.profile_report()

In [ ]:
df_skills_clean.profile_report()